In [ ]:
# Install required packages
!pip install transformers datasets faiss-cpu sentence-transformers

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from typing import List, Tuple

class RAGSystem:
    def __init__(self, retriever_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                 qa_model_name: str = "deepset/roberta-base-squad2"):
        # Initialize embedding model for retrieval
        self.embedding_model = SentenceTransformer(retriever_model_name)

        # Initialize QA model and tokenizer
        self.qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
        self.qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

        # Initialize FAISS index
        self.index = None
        self.documents = []

    def add_documents(self, documents: List[str]):
        """Add documents to the RAG system"""
        self.documents = documents

        # Create embeddings for documents
        embeddings = self.embedding_model.encode(documents)

        # Initialize FAISS index
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)

        # Add embeddings to FAISS index
        self.index.add(embeddings.astype('float32'))

    def retrieve(self, query: str, k: int = 3) -> List[str]:
        """Retrieve relevant documents for a query"""
        # Get query embedding
        query_embedding = self.embedding_model.encode([query])

        # Search in FAISS index
        distances, indices = self.index.search(query_embedding.astype('float32'), k)

        # Return retrieved documents
        return [self.documents[idx] for idx in indices[0]]

    def answer_question(self, question: str, context: str) -> Tuple[str, float]:
        """Extract answer from context using QA model"""
        # Tokenize input
        inputs = self.qa_tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)

        # Get model outputs
        with torch.no_grad():
            outputs = self.qa_model(**inputs)

        # Get answer span
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits)

        # Convert tokens to answer string
        tokens = self.qa_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        answer = self.qa_tokenizer.convert_tokens_to_string(tokens[answer_start:answer_end + 1])

        # Calculate confidence score
        confidence = float(torch.max(outputs.start_logits) + torch.max(outputs.end_logits)) / 2

        return answer, confidence

    def query(self, question: str, k: int = 3) -> List[Tuple[str, float]]:
        """Full RAG pipeline: retrieve documents and answer question"""
        # Retrieve relevant documents
        relevant_docs = self.retrieve(question, k)

        # Get answers from each document
        results = []
        for doc in relevant_docs:
            answer, confidence = self.answer_question(question, doc)
            results.append((answer, confidence))

        # Sort by confidence
        results.sort(key=lambda x: x[1], reverse=True)
        return results

# Example usage
def main():
    # Sample documents
    documents = [
        "The Earth is the third planet from the Sun and the only astronomical object known to harbor life.",
        "Python is a high-level, interpreted programming language created by Guido van Rossum.",
        "Machine learning is a subset of artificial intelligence that focuses on data and algorithms.",
    ]

    # Initialize RAG system
    rag = RAGSystem()

    # Add documents
    rag.add_documents(documents)

    # Test query
    question = "What is Python?"
    results = rag.query(question)

    print(f"Question: {question}")
    print("\nAnswers (sorted by confidence):")
    for answer, confidence in results:
        print(f"- Answer: {answer}")
        print(f"  Confidence: {confidence:.2f}\n")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Question: What is Python?

Answers (sorted by confidence):
- Answer:  a high-level, interpreted programming language
  Confidence: 6.72

- Answer: <s>
  Confidence: 3.23

- Answer: <s>
  Confidence: 2.78

